In [2]:
using Revise
using Turing, MCMCChains
using XLSX
using DataFrames, TimeSeries
using StatsPlots
using LinearAlgebra
using CSV
using StatsBase
include("utils.jl")
include("financial_bvar_temp.jl")
using .FinancialBVAR

In [3]:
df = DataFrame(XLSX.readtable("usa_var_data.xlsx", "Data", infer_eltypes=true))
data_source = collapse(TimeArray(df; timestamp = :Date), Dates.quarter, last)

292×11 TimeArray{Float64, 2, Date, Matrix{Float64}} 1951-12-01 to 2024-07-01
┌────────────┬──────────┬───────────┬─────────────┬──────────┬────────┬─────────
│            │ Yeld_10Y │ Price_10Y │ Stock_Price │ Dividend │ E      │ CAPE   ⋯
├────────────┼──────────┼───────────┼─────────────┼──────────┼────────┼─────────
│ 1951-12-01 │  0.02513 │   104.098 │       24.19 │  1.41333 │   2.44 │ 12.527 ⋯
│ 1952-03-01 │  0.02505 │   104.824 │       23.74 │     1.43 │    2.4 │ 12.242 ⋯
│ 1952-06-01 │  0.02415 │   106.295 │       25.08 │     1.45 │   2.34 │ 12.669 ⋯
│ 1952-09-01 │  0.02685 │   104.468 │       24.26 │  1.43667 │   2.36 │ 12.131 ⋯
│ 1952-12-01 │  0.02547 │   106.402 │       26.18 │     1.41 │    2.4 │ 13.010 ⋯
│ 1953-03-01 │  0.02735 │   105.373 │       24.71 │  1.41333 │   2.43 │ 12.163 ⋯
│ 1953-06-01 │  0.02987 │    103.86 │       24.29 │     1.42 │   2.51 │ 11.750 ⋯
│ 1953-09-01 │  0.02636 │    107.81 │       23.97 │     1.43 │   2.55 │ 11.391 ⋯
│     ⋮      │    ⋮     │     ⋮ 

In [4]:
colnames(data_source)

11-element Vector{Symbol}:
 :Yeld_10Y
 :Price_10Y
 :Stock_Price
 :Dividend
 :E
 :CAPE
 :T_Bill
 :CPI
 :Dividend_Yeld
 :Total_Stock_2
 :Total_Stock

In [5]:
returns = percentchange(data_source[:Total_Stock, :Price_10Y], :log)
returns = merge(returns, log1p.(data_source[:T_Bill])./4)
cpi = percentchange(data_source[:CPI], :log)
real_returns = returns .- cpi
term = data_source[:Yeld_10Y] ./4 .- data_source[:T_Bill] ./4
dy = data_source[:Dividend_Yeld] #log.(data_source[:Dividend_Yeld])
data = merge(real_returns, dy, term, cpi)

291×6 TimeArray{Float64, 2, Date, Matrix{Float64}} 1952-03-01 to 2024-07-01
┌────────────┬─────────────────┬───────────────┬─────────────┬───────────────┬──
│            │ Total_Stock_CPI │ Price_10Y_CPI │ T_Bill_CPI  │ Dividend_Yeld │ ⋯
├────────────┼─────────────────┼───────────────┼─────────────┼───────────────┼──
│ 1952-03-01 │      1.98335e-5 │     0.0107337 │  0.00772445 │     0.0602359 │ ⋯
│ 1952-06-01 │       0.0581411 │    0.00263202 │ -0.00708528 │      0.057815 │ ⋯
│ 1952-09-01 │      -0.0185027 │    -0.0173351 │  0.00423886 │     0.0592197 │ ⋯
│ 1952-12-01 │       0.0932838 │     0.0220972 │   0.0089235 │     0.0538579 │ ⋯
│ 1953-03-01 │      -0.0437093 │   -0.00972202 │  0.00497517 │     0.0571967 │ ⋯
│ 1953-06-01 │      -0.0100139 │    -0.0219485 │ -0.00227055 │     0.0584603 │ ⋯
│ 1953-09-01 │     -0.00574595 │     0.0298874 │ -0.00299956 │     0.0596579 │ ⋯
│ 1953-12-01 │        0.078226 │     0.0248173 │  0.00767892 │     0.0572141 │ ⋯
│     ⋮      │        ⋮        │ 

In [6]:
CSV.write("usa_data_transformed.csv", data)

"usa_data_transformed.csv"

In [7]:
start = Date(1993,03,01)
end_d = Date(2023, 12, 31)
s_data = to(from(data, start),end_d)

124×6 TimeArray{Float64, 2, Date, Matrix{Float64}} 1993-03-01 to 2023-12-01
┌────────────┬─────────────────┬───────────────┬─────────────┬───────────────┬──
│            │ Total_Stock_CPI │ Price_10Y_CPI │ T_Bill_CPI  │ Dividend_Yeld │ ⋯
├────────────┼─────────────────┼───────────────┼─────────────┼───────────────┼──
│ 1993-03-01 │       0.0150658 │     0.0543577 │ -0.00250148 │     0.0281965 │ ⋯
│ 1993-06-01 │       0.0136547 │     0.0292271 │  0.00478562 │     0.0279908 │ ⋯
│ 1993-09-01 │       0.0342312 │       0.03506 │ -0.00169417 │     0.0270317 │ ⋯
│ 1993-12-01 │       0.0226641 │    -0.0219636 │  0.00410946 │     0.0266883 │ ⋯
│ 1994-03-01 │      -0.0571526 │    -0.0622961 │ 0.000425924 │     0.0285162 │ ⋯
│ 1994-06-01 │      0.00958316 │    -0.0293652 │  0.00338014 │     0.0285113 │ ⋯
│ 1994-09-01 │        0.026626 │   -0.00884948 │  0.00390608 │     0.0280574 │ ⋯
│ 1994-12-01 │      0.00488791 │  -0.000941161 │  0.00828514 │     0.0283289 │ ⋯
│     ⋮      │        ⋮        │ 

In [8]:
returns_summarystats(to(from(returns, start),end_d), 4)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock,0.144099,0.782737,0.0963943,0.261158,-1.01471,-0.845921,-0.0423317,0.0953652,1.77521,0.668946,0.142769
Price_10Y,0.0832445,0.569087,0.044665,0.156403,-0.289949,0.371142,-0.0740101,0.0442422,0.221941,0.536551,0.0280389
T_Bill,0.0101058,0.0582689,0.0227848,0.045069,9.9995e-5,0.292382,0.00149888,0.968883,-1.50635,2.25464,0.0172994


In [9]:
returns_summarystats(s_data, 1)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.0717569,0.193813,0.0178775,0.0610642,-0.238141,-0.699168,-0.0145302,0.0849045,1.2878,0.24914,0.0283261
Price_10Y_CPI,0.0448954,0.167664,0.0049452,0.0347394,-0.0960374,0.432308,-0.0258376,0.0666902,0.789709,0.110149,0.00203425
T_Bill_CPI,0.00893438,0.025467,-0.000524846,0.00528326,-0.026825,-0.318287,-0.00503521,0.302165,0.492507,-0.0587446,0.000505661
Dividend_Yeld,0.00435984,0.0323637,0.0189393,0.0207695,0.0112582,0.765491,0.0164025,0.918403,0.717749,4.34404,0.0185857
Yeld_10Y_T_Bill,0.00293746,0.0095,0.00388508,0.006225,-0.0034,-0.236784,0.0017125,0.874579,-0.53084,1.3226,0.00395
CPI,0.00774615,0.0279226,0.00622104,0.00985738,-0.025392,-0.34528,0.00268687,0.0727647,2.47992,0.803114,0.00575376


In [10]:
returns_summarystats(s_data, 4)

,std,max,mean,p75th,min,skewness,p25th,autocor,kurtosis,sr,median
Total_Stock_CPI,0.143514,0.775253,0.0715101,0.244257,-0.952565,-0.699168,-0.0581207,0.0849045,1.2878,0.498281,0.113304
Price_10Y_CPI,0.0897907,0.670655,0.0197808,0.138958,-0.38415,0.432308,-0.10335,0.0666902,0.789709,0.220299,0.00813698
T_Bill_CPI,0.0178688,0.101868,-0.00209938,0.021133,-0.1073,-0.318287,-0.0201408,0.302165,0.492507,-0.117489,0.00202265
Dividend_Yeld,0.00871968,0.129455,0.0757573,0.0830779,0.0450328,0.765491,0.06561,0.918403,0.717749,8.68808,0.0743426
Yeld_10Y_T_Bill,0.00587492,0.038,0.0155403,0.0249,-0.0136,-0.236784,0.00685,0.874579,-0.53084,2.6452,0.0158
CPI,0.0154923,0.11169,0.0248842,0.0394295,-0.101568,-0.34528,0.0107475,0.0727647,2.47992,1.60623,0.0230151


In [11]:
vmodel = VARModel(s_data)
model_summary(vmodel)

,const,Total_Stock_CPI,Price_10Y_CPI,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill,CPI,std
Total_Stock_CPI,-0.0295,0.1152,0.0788,-0.8405,4.0279,-5.652,-1.5289,0.07
Price_10Y_CPI,-0.0077,-0.1285,-0.0358,2.8038,-0.488,3.675,1.779,0.0427
T_Bill_CPI,-0.0122,-0.0249,-0.0146,1.0719,0.3155,0.119,1.0222,0.0073
Dividend_Yeld,0.0012,-0.003,-0.0016,0.002,0.9069,0.0919,0.0168,0.0015
Yeld_10Y_T_Bill,0.0003,0.0017,0.0066,-0.0515,0.0091,0.887,-0.0329,0.0012
CPI,0.0118,0.0262,0.0089,-0.0735,-0.3003,-0.0871,-0.0139,0.0075


,Total_Stock_CPI,Price_10Y_CPI,T_Bill_CPI,Dividend_Yeld,Yeld_10Y_T_Bill,CPI
Total_Stock_CPI,1.0,-0.16,0.03,-0.91,0.02,-0.01
Price_10Y_CPI,-0.16,1.0,0.44,0.23,-0.58,-0.49
T_Bill_CPI,0.03,0.44,1.0,0.16,-0.14,-0.99
Dividend_Yeld,-0.91,0.23,0.16,1.0,-0.07,-0.17
Yeld_10Y_T_Bill,0.02,-0.58,-0.14,-0.07,1.0,0.07
CPI,-0.01,-0.49,-0.99,-0.17,0.07,1.0


In [12]:
bmodel = NormalWishartBVAR(s_data)
sample_posterior!(bmodel,10000, 2000)
posterior_summary(bmodel)

"Total_Stock_CPI coefficients"

Quantiles
       parameters       2.5%     25.0%     50.0%     75.0%     97.5% 
           Symbol    Float64   Float64   Float64   Float64   Float64 

            const    -0.0976   -0.0523   -0.0285   -0.0057    0.0397
  Total_Stock_CPI    -0.0689    0.0487    0.1144    0.1785    0.3026
    Price_10Y_CPI    -0.2660   -0.0401    0.0784    0.1923    0.4276
       T_Bill_CPI    -3.9014   -1.9048   -0.8643    0.2137    2.2030
    Dividend_Yeld     0.6261    2.8615    3.9956    5.1446    7.3632
  Yeld_10Y_T_Bill   -11.2454   -7.5941   -5.6556   -3.6177    0.2533
              CPI    -5.0192   -2.7842   -1.5505   -0.3574    2.0022


"Price_10Y_CPI coefficients"

Quantiles
       parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
           Symbol   Float64   Float64   Float64   Float64   Float64 

            const   -0.0487   -0.0221   -0.0079    0.0069    0.0350
  Total_Stock_CPI   -0.2418   -0.1669   -0.1282   -0.0897   -0.0172
    Price_10Y_CPI   -0.2465   -0.1077   -0.0352    0.0344    0.1780
       T_Bill_CPI    0.9280    2.1791    2.8188    3.4577    4.6800
    Dividend_Yeld   -2.5277   -1.1973   -0.5053    0.1838    1.5915
  Yeld_10Y_T_Bill    0.1213    2.4640    3.6467    4.9225    7.2741
              CPI   -0.3205    1.0656    1.7866    2.5065    3.9650


"T_Bill_CPI coefficients"

...

Quantiles
       parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
           Symbol   Float64   Float64   Float64   Float64   Float64 

            const   -0.0009   -0.0001    0.0003    0.0007    0.0015
  Total_Stock_CPI   -0.0016    0.0006    0.0017    0.0028    0.0049
    Price_10Y_CPI    0.0005    0.0045    0.0066    0.0086    0.0126
       T_Bill_CPI   -0.1056   -0.0697   -0.0515   -0.0331    0.0016
    Dividend_Yeld   -0.0492   -0.0107    0.0094    0.0289    0.0685
  Yeld_10Y_T_Bill    0.7840    0.8514    0.8867    0.9219    0.9902
              CPI   -0.0950   -0.0543   -0.0331   -0.0118    0.0281


"CPI coefficients"

Quantiles
       parameters      2.5%     25.0%     50.0%     75.0%     97.5% 
           Symbol   Float64   Float64   Float64   Float64   Float64 

            const    0.0043    0.0092    0.0118    0.0143    0.0191
  Total_Stock_CPI    0.0061    0.0192    0.0261    0.0328    0.0464
    Price_10Y_CPI   -0.0283   -0.0042    0.0088    0.0214    0.0470
       T_Bill_CPI   -0.4077   -0.1843   -0.0751    0.0369    0.2519
    Dividend_Yeld   -0.6625   -0.4207   -0.2988   -0.1750    0.0529
  Yeld_10Y_T_Bill   -0.7218   -0.3043   -0.0866    0.1258    0.5546
              CPI   -0.3921   -0.1409   -0.0138    0.1142    0.3643


"correlation matrix"

Quantiles
                       parameters      2.5%     25.0%     50.0%     75.0%      ⋯
                           Symbol   Float64   Float64   Float64   Float64   Fl ⋯

  Total_Stock_CPI_Total_Stock_CPI    0.0633    0.0686    0.0718    0.0751    0 ⋯
    Total_Stock_CPI_Price_10Y_CPI   -0.3299   -0.2187   -0.1567   -0.0929    0 ⋯
       Total_Stock_CPI_T_Bill_CPI   -0.1579   -0.0347    0.0309    0.0927    0 ⋯
    Total_Stock_CPI_Dividend_Yeld   -0.9383   -0.9223   -0.9122   -0.9007   -0 ⋯
  Total_Stock_CPI_Yeld_10Y_T_Bill   -0.1657   -0.0436    0.0199    0.0829    0 ⋯
              Total_Stock_CPI_CPI   -0.1862   -0.0703   -0.0084    0.0566    0 ⋯
    Price_10Y_CPI_Total_Stock_CPI   -0.3299   -0.2187   -0.1567   -0.0929    0 ⋯
      Price_10Y_CPI_Price_10Y_CPI    0.0386    0.0419    0.0438    0.0459    0 ⋯
         Price_10Y_CPI_T_Bill_CPI    0.2760    0.3848    0.4380    0.4889    0 ⋯
      Price_10Y_CPI_Dividend_Yeld    0.0435    0.1670    0.2286    0.2875    0 ⋯
    Price_10Y_CPI

In [13]:
vmodel.Y[end,:]

6-element Vector{Float64}:
  0.12159459085777868
  0.06515539231011935
  0.010346584402051042
  0.014635185560818083
 -0.0034000000000000002
  0.002421733143949112

In [20]:
assets_names = colnames(s_data)
scenarios = simulate(vmodel,60)

for n in 1:6
    s_ann = annualise(scenarios[n,:,2:end],4)
    print_percentiles(s_ann,[0.05, 0.25, 0.5, 0.75, 0.95], string(assets_names[n]))
end

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.0914264,-0.166235,-0.181467,-0.191516,-0.192916,-0.19439,-0.198993,-0.201448,-0.198589,-0.200832,-0.190478,-0.189801,-0.192296,-0.192602,-0.190796
0.25,0.0412362,-0.0236145,-0.0374463,-0.0446366,-0.0446839,-0.0489995,-0.0511952,-0.0524132,-0.0500087,-0.0520655,-0.0461397,-0.0458595,-0.0445811,-0.0515806,-0.0470831
0.5,0.132245,0.0724707,0.0647173,0.0571982,0.0572795,0.0497614,0.0492833,0.04956,0.0529362,0.0500403,0.0540592,0.0544128,0.0570557,0.0525163,0.0572094
0.75,0.224484,0.169093,0.164478,0.15908,0.157189,0.151336,0.151533,0.151376,0.152658,0.149887,0.153566,0.153371,0.155885,0.158153,0.158078
0.95,0.355794,0.310921,0.315848,0.304942,0.302085,0.297941,0.292223,0.297811,0.295309,0.293216,0.298144,0.295586,0.300733,0.301216,0.30531


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.152905,-0.134035,-0.131935,-0.13193,-0.137293,-0.137305,-0.139585,-0.136986,-0.139326,-0.142033,-0.143561,-0.141921,-0.141714,-0.145105,-0.143401
0.25,-0.0750069,-0.0499049,-0.0478728,-0.0482582,-0.0490058,-0.0488786,-0.0488384,-0.0503377,-0.0517301,-0.0523769,-0.0543836,-0.0551685,-0.0535549,-0.0554314,-0.0535267
0.5,-0.0220315,0.00782391,0.0114069,0.0112654,0.0126324,0.0130171,0.0134171,0.0102288,0.00952412,0.00900515,0.00728295,0.00735701,0.00605844,0.00643501,0.00785457
0.75,0.0321482,0.067152,0.0694977,0.0723966,0.0730979,0.072517,0.0746239,0.0717186,0.0709119,0.070766,0.0675765,0.0704026,0.0691781,0.0694814,0.0695819
0.95,0.111102,0.149867,0.159578,0.159261,0.159994,0.162447,0.158423,0.158941,0.159938,0.159732,0.157236,0.156409,0.155409,0.155768,0.156894


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.0122728,-0.0152859,-0.0206906,-0.0252782,-0.0287387,-0.0309765,-0.0339041,-0.0352543,-0.0369301,-0.0384938,-0.0397396,-0.0402267,-0.0412799,-0.0417004,-0.0426488
0.25,0.00233276,0.00113836,-0.00234373,-0.00511822,-0.00764806,-0.0100374,-0.0120336,-0.0135493,-0.0147945,-0.0158868,-0.0172012,-0.0178596,-0.0180778,-0.0185931,-0.0191592
0.5,0.0123217,0.0130151,0.0106676,0.00851559,0.00635942,0.00483137,0.00338741,0.00177402,0.000949515,-0.000133466,-0.00100922,-0.00160584,-0.00215037,-0.00289786,-0.00337179
0.75,0.0224104,0.0246394,0.0237308,0.0223629,0.0209869,0.0195753,0.0181151,0.0172023,0.0163633,0.0157846,0.0147823,0.0144533,0.013843,0.0131752,0.0124388
0.95,0.0367786,0.0416492,0.0426339,0.0425995,0.0420659,0.0415,0.0399908,0.0404592,0.0389228,0.0378584,0.0376798,0.0373556,0.036359,0.0367706,0.035942


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,0.0418032,0.0342584,0.0315512,0.0308052,0.0308484,0.0312621,0.0318132,0.0330266,0.0329199,0.0339054,0.0345881,0.0358735,0.0363445,0.0368254,0.0366138
0.25,0.0489007,0.0450844,0.0442798,0.0443417,0.0451311,0.0460811,0.0471428,0.0480644,0.0493596,0.0504924,0.0513476,0.0517668,0.0523005,0.0529059,0.0534196
0.5,0.0541469,0.0527731,0.0529826,0.0538712,0.0549246,0.0562572,0.0577124,0.0589843,0.06013,0.0616925,0.0624017,0.0629434,0.0635409,0.0643757,0.0648972
0.75,0.059182,0.0604179,0.0615982,0.0629662,0.0645596,0.0662496,0.0677841,0.0695405,0.0711103,0.0725456,0.0735334,0.074104,0.0748374,0.0754717,0.0763146
0.95,0.0663978,0.0717908,0.0747278,0.077054,0.0790366,0.081468,0.0829245,0.0846879,0.0862147,0.0883786,0.0894052,0.0901737,0.0907143,0.0918042,0.0924465


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,-0.0177212,-0.0192138,-0.0188375,-0.0184353,-0.0175008,-0.0168477,-0.016305,-0.0157462,-0.0149436,-0.0141753,-0.013663,-0.0133664,-0.0129311,-0.0125349,-0.0121536
0.25,-0.0123183,-0.0109448,-0.00941958,-0.00805774,-0.00676174,-0.0054887,-0.00447765,-0.00357165,-0.00249418,-0.00163205,-0.00126055,-0.000846119,-0.00019663,0.000115481,0.000721703
0.5,-0.00850751,-0.00528044,-0.00279378,-0.000746214,0.000894447,0.00235762,0.00375237,0.00493735,0.00601099,0.00688534,0.00762643,0.00830094,0.00883743,0.00920825,0.00949763
0.75,-0.00479675,0.000458916,0.0037322,0.00653359,0.00856809,0.0101896,0.0118105,0.0131283,0.0145392,0.0155686,0.0164252,0.0171395,0.017701,0.0181085,0.0184148
0.95,0.000821012,0.00858724,0.0134927,0.0169763,0.0196808,0.0217009,0.023434,0.025454,0.0267256,0.0279862,0.0290238,0.0294792,0.0301654,0.0308966,0.0313629


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.05,0.00980763,0.00584449,0.00520158,0.00465955,0.00391366,0.00367096,0.00240261,0.00254381,0.00198644,0.00172432,0.000818034,0.00150955,0.00109562,0.000713815,0.000971484
0.25,0.0252725,0.021745,0.0208137,0.0207208,0.0195527,0.0194431,0.0190459,0.0187427,0.0180352,0.0178187,0.0176163,0.0175372,0.0173844,0.0170534,0.017103
0.5,0.0360876,0.0327867,0.0319794,0.0318503,0.0312974,0.0307897,0.0304805,0.0297633,0.0294004,0.0291274,0.0289701,0.0288696,0.0284058,0.0284343,0.0281106
0.75,0.0467636,0.043635,0.0432832,0.0425188,0.0422309,0.0419661,0.0417033,0.0408988,0.0405835,0.0401701,0.0405339,0.0399315,0.0398113,0.0397032,0.0391849
0.95,0.0624036,0.059119,0.059012,0.0585284,0.0582956,0.0573036,0.0575366,0.057251,0.0568668,0.0559815,0.0563719,0.0562913,0.0557798,0.055398,0.0554357


In [25]:
scenarios_b = simulate(bmodel,60, 10)
for n in 1:6
    s_ann = annualise(scenarios_b[n,:,:],4)
    print_percentiles(s_ann,[0.02, 0.25, 0.5, 0.75, 0.98], string(assets_names[n]))
end

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,0.0164852,-0.482036,-0.585131,-0.630199,-0.65057,-0.667927,-0.688733,-0.697536,-0.709056,-0.722662,-0.740901,-0.75988,-0.76692,-0.776805,-0.8015
0.25,0.162446,-0.0722056,-0.111875,-0.126213,-0.13447,-0.137593,-0.141874,-0.143344,-0.142986,-0.145602,-0.147624,-0.149344,-0.151635,-0.151461,-0.152935
0.5,0.224026,0.0797717,0.0675319,0.0647153,0.0613893,0.0613508,0.0601045,0.0639357,0.0648276,0.0633084,0.0636855,0.0650361,0.0653242,0.0639349,0.0652395
0.75,0.292144,0.239281,0.249309,0.253949,0.260752,0.264376,0.268988,0.272039,0.275484,0.278278,0.280194,0.284348,0.28524,0.286881,0.289622
0.98,0.476112,0.682707,0.753002,0.779514,0.802921,0.820636,0.852279,0.871937,0.89538,0.919205,0.961725,0.983947,1.01062,1.03502,1.06682


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.122657,-0.473723,-0.559674,-0.598516,-0.624504,-0.640631,-0.653512,-0.668172,-0.67833,-0.682136,-0.687101,-0.701168,-0.710851,-0.715499,-0.731081
0.25,0.00183517,-0.125069,-0.149547,-0.164506,-0.175764,-0.183383,-0.188648,-0.19402,-0.197773,-0.201762,-0.205741,-0.208889,-0.210466,-0.213195,-0.214535
0.5,0.049256,0.00774168,0.0113524,0.0109409,0.00895262,0.00869339,0.00901476,0.00659612,0.00492239,0.00423338,0.00191487,0.00258212,0.00124335,0.000703434,0.000251564
0.75,0.0929802,0.136624,0.172809,0.188323,0.195498,0.200398,0.203158,0.207547,0.208782,0.209721,0.209987,0.211131,0.212744,0.213286,0.215749
0.98,0.198708,0.47259,0.57831,0.620716,0.642014,0.660491,0.675681,0.678091,0.678528,0.681811,0.696524,0.701375,0.701774,0.70999,0.712244


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.0334678,-0.125872,-0.173581,-0.206117,-0.234731,-0.25923,-0.281569,-0.29836,-0.317022,-0.334889,-0.348922,-0.368494,-0.386821,-0.404643,-0.42407
0.25,0.0024969,-0.0296544,-0.045751,-0.0568102,-0.0660593,-0.072716,-0.0778585,-0.0822786,-0.0860246,-0.0889832,-0.0913763,-0.0936775,-0.0956993,-0.0968766,-0.0981973
0.5,0.0189541,0.0131788,0.0107673,0.00890981,0.00639776,0.00595217,0.00485545,0.003728,0.00322252,0.00211359,0.00148076,0.00186077,0.00150115,0.00114263,0.00135534
0.75,0.0352632,0.0561196,0.0677952,0.0748645,0.0802122,0.0842968,0.0877692,0.0909174,0.0934749,0.0958135,0.0969084,0.0984,0.100089,0.100804,0.101991
0.98,0.0719413,0.155352,0.198465,0.229997,0.253875,0.277427,0.299879,0.320772,0.339773,0.360441,0.383788,0.406156,0.431094,0.456412,0.485307


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,0.0167084,-0.0298668,-0.0553163,-0.0746374,-0.0925608,-0.108949,-0.127351,-0.146278,-0.165369,-0.185188,-0.20745,-0.23033,-0.255496,-0.282115,-0.306095
0.25,0.043234,0.0272107,0.0205205,0.0166624,0.0143044,0.011971,0.0102379,0.0090407,0.00736603,0.00620867,0.00581208,0.00450793,0.00361512,0.00263499,0.00177915
0.5,0.0553456,0.0530027,0.0529735,0.0538664,0.0554516,0.0562183,0.0574206,0.0583495,0.0593186,0.0597111,0.0601048,0.0609256,0.0612435,0.0617463,0.0622037
0.75,0.0674813,0.0784836,0.0846827,0.0895948,0.0941435,0.0984463,0.101787,0.10471,0.107243,0.109188,0.111165,0.112902,0.11477,0.116223,0.117529
0.98,0.09381,0.133587,0.154746,0.171002,0.184962,0.197924,0.21139,0.225214,0.238797,0.251593,0.267291,0.277991,0.292556,0.306962,0.324723


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,-0.0481616,-0.0857382,-0.105111,-0.12137,-0.135686,-0.150027,-0.165885,-0.18316,-0.20077,-0.219684,-0.241729,-0.265108,-0.288269,-0.311736,-0.337985
0.25,-0.0223152,-0.030683,-0.0335449,-0.0358878,-0.0373826,-0.0388076,-0.0400724,-0.0415724,-0.0431537,-0.044327,-0.0449925,-0.0456991,-0.0469876,-0.0476059,-0.0479214
0.5,-0.0101964,-0.00592789,-0.00313056,-0.00105572,0.000414792,0.00152465,0.00291836,0.00332886,0.00367574,0.00460775,0.00484033,0.00532705,0.00591959,0.00607638,0.00642981
0.75,0.00169682,0.0183759,0.0263822,0.0319104,0.0362579,0.0403679,0.043273,0.0457398,0.0481579,0.0502275,0.0515754,0.0533406,0.0548185,0.0559571,0.0564203
0.98,0.0276384,0.0714435,0.0921169,0.1072,0.119076,0.132202,0.144373,0.156531,0.168707,0.17989,0.193003,0.204985,0.216475,0.230532,0.243261


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.02,0.00498865,-0.0168493,-0.0287387,-0.037657,-0.0449392,-0.049611,-0.0568062,-0.0620419,-0.0685096,-0.07485,-0.0810933,-0.0867232,-0.0927015,-0.0990773,-0.106204
0.25,0.0220882,0.0180285,0.0151265,0.0129887,0.0115938,0.0101401,0.00906979,0.00822724,0.00720201,0.00652478,0.00591372,0.00527317,0.0047882,0.00429858,0.00398928
0.5,0.029896,0.0319043,0.0314969,0.030983,0.0308086,0.0305736,0.0300637,0.0299175,0.0295675,0.0296569,0.0293814,0.0292553,0.0291492,0.0290456,0.0289106
0.75,0.0379254,0.0466873,0.0491817,0.0504423,0.0516754,0.0524143,0.0531641,0.0537984,0.054363,0.0548582,0.055292,0.0556951,0.0561294,0.056721,0.0569466
0.98,0.0563924,0.0875306,0.100764,0.110227,0.1187,0.126059,0.135282,0.143903,0.153999,0.162532,0.173153,0.184809,0.196272,0.209067,0.224069


In [30]:
# annualised compounded returns for 1, 5, 10, 15 years
for n in 1:3
    cum_ret = cumsum(scenarios[n,:,2:end]; dims=2)
    print_percentiles(expm1.(cum_ret[:,[4,20,40,60]] ./ [1 5 10 15]),[.02, .1, .25, .5, .75, .9, .98], string(assets_names[n]))    
end

,1,2,3,4
0.02,-0.137414,-0.022755,0.00127509,0.0140085
0.1,-0.0400053,0.0160845,0.0247501,0.0310375
0.25,0.0420982,0.0462731,0.0431344,0.0455853
0.5,0.141388,0.0794099,0.0651323,0.0620945
0.75,0.251677,0.114444,0.0874368,0.0786453
0.9,0.359518,0.146683,0.106862,0.0939325
0.98,0.509207,0.191123,0.133844,0.114542


,1,2,3,4
0.02,-0.169453,-0.0397351,-0.0185439,-0.0130611
0.1,-0.116099,-0.0236589,-0.00854625,-0.00516369
0.25,-0.072263,-0.0103109,-0.00102781,0.000788368
0.5,-0.0217906,0.00449305,0.00776574,0.00750977
0.75,0.0326705,0.0195271,0.0162989,0.014405
0.9,0.0839054,0.0334738,0.0243995,0.0207166
0.98,0.154702,0.0515081,0.0347305,0.0282654


,1,2,3,4
0.02,-0.0184106,-0.0115214,-0.0174087,-0.0204694
0.1,-0.00704473,-0.00331261,-0.00857073,-0.0115251
0.25,0.00233549,0.0031064,-0.00162631,-0.00453782
0.5,0.012398,0.0103206,0.00618258,0.00342495
0.75,0.0226634,0.0176406,0.0143673,0.0113517
0.9,0.0320464,0.0239761,0.0211695,0.018578
0.98,0.0439073,0.0327511,0.0302092,0.0275474


In [37]:
for n in 1:3
    s_ann = expm1.(annualise(scenarios[n,:,2:end],4))
    print_percentiles(s_ann,[.1, .3, .5, .7, .9], string(assets_names[n]))
end


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.1,-0.0400053,-0.107446,-0.120436,-0.129755,-0.12846,-0.130557,-0.134324,-0.134422,-0.13297,-0.134843,-0.126646,-0.127074,-0.126358,-0.131253,-0.128533
0.3,0.0624989,-0.00256743,-0.0143192,-0.0225665,-0.0222203,-0.0274237,-0.0282532,-0.028938,-0.0266919,-0.029522,-0.0236193,-0.0227116,-0.0234156,-0.0268994,-0.0235751
0.5,0.141388,0.0751613,0.0668574,0.0588657,0.0589517,0.0510203,0.0505179,0.0508086,0.0543624,0.0513135,0.0555471,0.0559204,0.0587147,0.0539198,0.0588775
0.7,0.226778,0.159129,0.151716,0.145356,0.14414,0.136721,0.136568,0.13773,0.139523,0.136058,0.140017,0.139631,0.142272,0.144076,0.144403
0.9,0.359518,0.29294,0.290734,0.280331,0.282792,0.278342,0.272603,0.274026,0.272455,0.273275,0.274506,0.27128,0.280923,0.279482,0.285909


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.1,-0.116099,-0.0984382,-0.0955991,-0.0958971,-0.0969438,-0.0986077,-0.0994807,-0.0973523,-0.101223,-0.101536,-0.102411,-0.105023,-0.102498,-0.104129,-0.104396
0.3,-0.0617304,-0.0368347,-0.0341069,-0.0342434,-0.0343007,-0.0344488,-0.0339683,-0.0361904,-0.0376928,-0.0371718,-0.0396201,-0.0404101,-0.0394008,-0.0406726,-0.038842
0.5,-0.0217906,0.0078546,0.0114722,0.0113291,0.0127125,0.0131022,0.0135075,0.0102813,0.00956962,0.00904582,0.00730953,0.00738414,0.00607683,0.00645576,0.0078855
0.7,0.0193047,0.0559261,0.0582719,0.0607175,0.0608175,0.0607945,0.0629253,0.0594009,0.0579011,0.0583985,0.0560114,0.0575547,0.0558687,0.0577999,0.0572782
0.9,0.0839054,0.126042,0.13487,0.134222,0.136611,0.136181,0.137409,0.132982,0.135327,0.134869,0.132188,0.132151,0.130804,0.131021,0.132258


,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0.1,-0.00704473,-0.00903252,-0.0133539,-0.0180358,-0.0211077,-0.0231575,-0.0254042,-0.0266992,-0.0278928,-0.0293513,-0.0307949,-0.0318473,-0.0320811,-0.0326381,-0.032998
0.3,0.00457754,0.00413171,0.000522394,-0.00203298,-0.00432912,-0.00682214,-0.00846321,-0.0101659,-0.0113134,-0.0123464,-0.0134365,-0.0143228,-0.0143045,-0.015203,-0.015575
0.5,0.012398,0.0131001,0.0107247,0.00855196,0.00637968,0.00484306,0.00339315,0.00177559,0.000949966,-0.000133457,-0.00100871,-0.00160455,-0.00214806,-0.00289366,-0.00336611
0.7,0.0203641,0.0221872,0.0208802,0.0191684,0.0179091,0.0165328,0.0150908,0.0136816,0.0130552,0.0122828,0.0112368,0.0109501,0.0104883,0.00955272,0.00900319
0.9,0.0320464,0.0361745,0.0362976,0.035441,0.0347669,0.0337327,0.032295,0.0316074,0.0307252,0.0302598,0.0295028,0.0290255,0.0280094,0.0284006,0.0277153


In [38]:
kurtosis(rand(Normal(0,1),1000_000))

0.006141041016670901